<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/SQL/Python_SQL_%EC%97%B0%EB%8F%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 1. 파이썬과 MySQL 활용

- 파이썬과 MySQL을 연결해주는 모듈 : mysqlclient, PyMySQL
  - pip install mysqlclient
  - pip install PyMySQL



In [12]:
pip install mysqlclient

In [3]:
pip install PyMySQL

     |████████████████████████████████| 51kB 2.6MB/s 


In [ ]:
# mysqlclient 모듈 사용

import MySQLdb

conn = MySQLdb.connect(host = "127.0.0.1", user = "root", password = "1111", database = "world")
print(conn)
conn.close()

In [ ]:
# pymysql 모듈 사용

import pymysql

conn = pymysql.connect(host = "127.0.0.1", user = "root", password = "1111", database = "world")
print(conn)
conn.close()

In [ ]:
# mysqlclient 모듈을 사용해서 계속 실습하기

config = {"host" : "127.0.0.1", "user" : "root", "password" : "1111", "database" : "world"}

### (1) SELECT

In [ ]:
conn = MySQLdb.connect(**config)

sql = 'select * from city limit 10'

cur = conn.cursor()
cur.execute(sql)

# print(cur)
# for data in cur:
#     print(data)
#     print("%d %s %s %s %d"% data)

# for a,b,c,d,e in cur:
#     print(a,b,c,d,e)


result = list()
for data in cur:
    result.append(data)
       
# result = list(data for data in cur)    
    
conn.close()

print(result)

### (2) INSERT

In [ ]:
conn = MySQLdb.connect(**config)

sql = "create table if not exists tbltest(empno int, ename varchar(10), sal int, job varchar(10))"
# if not exists -> 테이블 중복 생성시 오류 방지

cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"
cur.execute(sql)
conn.commit()

conn.close()

In [ ]:
empno = 2
ename = 'miller'
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(" + str(empno) + ", '" + ename + "', " + str(sal) + ")"
# miller 에 따옴표를 해주어야 문자로 인식한다
print(sql)
cur.execute(sql)
conn.commit()

conn.close()

In [ ]:
empno = 3
ename = 'tom'
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()
# close를 안하면 서버에 부하가 가해진다

In [ ]:
try : 
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    empno = 4
    ename = 'jerry'
    sal = 2000
    
    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))

    cur.execute(sql, sql_data)
    conn.commit()
except Exception as e:
    print("error : ", e)
finally:
    conn.close()

In [ ]:
# 데이터 여러 개 입력하기

conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()

### (3) cursor 객체의 fetchall(), fetchmany(), fetchone()
- default는 fetchall()

In [ ]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from tbltest")
    
#     for i in cur.fetchall(10):
#         print(i)
        
#     for i in cur.fetchmany(5):
#         print(i)
        
    print(cur.fetchone())
    # fetchmany(1)보다 성능적으로 더 좋다
    
    
finally:
    conn.close()

### (4) DictCursor

- 필드명을 인덱스 대신 키로 사용

In [ ]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        print(i["empno"], i["ename"], i["sal"])

        
finally:
    conn.close()

### (5) 응용 실습1 : CSV파일을 DB에 저장

In [ ]:
'''
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
'''

In [ ]:
import csv

file = csv.reader(open("D:\Coding\Acorn\SQL\응용 실습용 CSV파일.csv"))
print(file)

# print(next(file))

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("delete from suppliers")
    conn.commit()
    #이미 테이블에 등록되어 있는 데이터들 전부 삭제하기
    
    for row in file:
#         print(row)
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
    conn.commit()
        
finally:
    conn.close()